# Train first set of one vs. rest (OVR) classifiers.

We train a set of classifiers that are used for computations of $ClassSim$ that is defined in our paper to estimate a similarity between classes.  
These classifiers are also used for classification tasks.  
Modules used in this notebook are defined in files in *models* directory.

## Set up

In [ ]:
import os
import sys

import numpy as np

import pandas as pd
import glob

import warnings
warnings.filterwarnings('ignore')

In [ ]:
BASE_MODEL_PATH="trained_model"
%mkdir -p $BASE_MODEL_PATH

In [ ]:
from models.modelutils import ModelCompiler

In [ ]:
compiler = ModelCompiler(BASE_MODEL_PATH)

In [ ]:
from models.processor import create_generators

TRAIN_DATAGEN, VALID_DATAGEN = create_generators()

Load category information and all image paths.

In [ ]:
from models.modelutils import dir2filedict, split_fdict

In [ ]:
fdict = dir2filedict("data")

In [ ]:
categories = sorted(fdict.keys())

Split data into {train, validation, test} datasets.

In [ ]:
trdict, testdict = split_fdict(fdict, test_size=0.2, random_state = 123)

In [ ]:
trdict, valdict = split_fdict(trdict, test_size=0.2, random_state = 456)

In [ ]:
valdict['clouds'][0:5]

Here is expected outputs.   
The output may be different if you create image urls yourself or exlude some files for GMM, but all the outputs in {*train.ipynb*, *classifier_similarity.ipynb*, *train_multiclass_classifier.ipynb*, *train_second.ipynb*} must be the same. 

['data/clouds/0678.jpeg',  
 'data/clouds/0701.jpeg',  
 'data/clouds/0431.jpeg',  
 'data/clouds/0033.jpeg',  
 'data/clouds/0290.jpeg']

## Train classifiers

In [ ]:
from models.one_vs_all import OneVsAllModelTrainer
from models.modelutils import split_files

In [ ]:
trainer = OneVsAllModelTrainer(TRAIN_DATAGEN, VALID_DATAGEN)

In [ ]:
def train_one_category(cat, epoch=5):
    model_path = "{}/model_{}".format(BASE_MODEL_PATH, cat)
    model = compiler.generate_compiled_model(model_path)
    
    trainer.set_model(model)
    trainer.set_savepath(model_path)
    
    true_train, false_train = split_files(cat, trdict)
    true_valid, false_valid = split_files(cat, valdict)
    
    trainer.set_dataset_files(true_train, false_train, true_valid, false_valid)
    trainer.train_model(eachepochs=epoch)

In [ ]:
for i in range(0, len(categories)):
    train_one_category(categories[i])